# EE399 HW4
## Ziwen


https://github.com/ZiwenLi0325/EE399.git

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from scipy import integrate
from mpl_toolkits.mplot3d import Axes3D
import torch
from torch import nn
from torch.optim import Adam
from scipy import integrate

In [4]:
dt = 0.01
T = 8
t = np.arange(0,T+dt,dt)
beta = 8/3
sigma = 10
rho = 28

# Define the Lorenz system
def lorenz_deriv(x_y_z, t0, sigma=sigma, beta=beta, rho=rho):
    x, y, z = x_y_z
    return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]

# Generate training data for rho=10, 28, 40
rhos = [10, 28, 40]
training_input = []
training_output = []

for rho in rhos:
    np.random.seed(123)
    x0 = -15 + 30 * np.random.random((100, 3))

    x_t = np.asarray([integrate.odeint(lorenz_deriv, x0_j, t, args=(sigma, beta, rho)) for x0_j in x0])
    
    for j in range(100):
        training_input.append(x_t[j,:-1,:])
        training_output.append(x_t[j,1:,:])

training_input = np.vstack(training_input)
training_output = np.vstack(training_output)

# Convert numpy arrays to PyTorch tensors
training_input_torch = torch.tensor(training_input, dtype=torch.float32)
training_output_torch = torch.tensor(training_output, dtype=torch.float32)

# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(3, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# Initialize the model and optimizer
model = Net()
optimizer = Adam(model.parameters())

# Define the loss function
criterion = nn.MSELoss()

# Train the network
for epoch in range(100):  # 100 epochs
    optimizer.zero_grad()   # zero the gradient buffers
    output = model(training_input_torch)
    loss = criterion(output, training_output_torch)
    loss.backward()
    optimizer.step()    # Does the update
    if epoch % 10 == 0:
        print('Epoch: {}, Loss: {:.5f}'.format(epoch, loss.item()))

# Now test for rho=17 and 35
rhos = [17, 35]
testing_input = []
testing_output = []

for rho in rhos:
    np.random.seed(123)
    x0 = -15 + 30 * np.random.random((100, 3))

    x_t = np.asarray([integrate.odeint(lorenz_deriv, x0_j, t, args=(sigma, beta, rho)) for x0_j in x0])
    
    for j in range(100):
        testing_input.append(x_t[j,:-1,:])
        testing_output.append(x_t[j,1:,:])

testing_input = np.vstack(testing_input)
testing_output = np.vstack(testing_output)

# Convert to PyTorch tensors
testing_input_torch = torch.tensor(testing_input, dtype=torch.float32)
testing_output_torch = torch.tensor(testing_output, dtype=torch.float32)

# Predict future states
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Do not calculate gradients
    predictions = model(testing_input_torch)

# Calculate the loss between the predictions and the true future states
test_loss = criterion(predictions, testing_output_torch)
print('Test loss:', test_loss.item())



Epoch: 0, Loss: 298.25491
Epoch: 10, Loss: 197.73874
Epoch: 20, Loss: 115.29742
Epoch: 30, Loss: 51.61908
Epoch: 40, Loss: 17.40350
Epoch: 50, Loss: 6.37015
Epoch: 60, Loss: 3.60739
Epoch: 70, Loss: 2.96787
Epoch: 80, Loss: 2.46544
Epoch: 90, Loss: 2.10413
Test loss: 1.4272576570510864


## Feed-forward Neural Network

In [5]:
# Define the Lorenz system
def lorenz_deriv(x_y_z, t0, sigma=sigma, beta=beta, rho=rho):
    x, y, z = x_y_z
    return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]

# Generate training data for rho=10, 28, 40
rhos = [10, 28, 40]
training_input = []
training_output = []

for rho in rhos:
    np.random.seed(123)
    x0 = -15 + 30 * np.random.random((100, 3))

    x_t = np.asarray([integrate.odeint(lorenz_deriv, x0_j, t, args=(sigma, beta, rho)) for x0_j in x0])
    
    for j in range(100):
        training_input.append(x_t[j,:-1,:])
        training_output.append(x_t[j,1:,:])

training_input = np.vstack(training_input)
training_output = np.vstack(training_output)

# Convert numpy arrays to PyTorch tensors
training_input_torch = torch.tensor(training_input, dtype=torch.float32)
training_output_torch = torch.tensor(training_output, dtype=torch.float32)

class FeedForwardNN(nn.Module):
    def __init__(self):
        super(FeedForwardNN, self).__init__()
        self.fc1 = nn.Linear(3, 50)  # Input dimension is 3 (for x, y, z)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 3)  # Output dimension is 3 (for x, y, z)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)  # No activation function here because the output can be any real number
    
# Initialize the model and optimizer
model = Net()
optimizer = Adam(model.parameters())

# Define the loss function
criterion = nn.MSELoss()

# Train the network
for epoch in range(100):  # 100 epochs
    optimizer.zero_grad()   # zero the gradient buffers
    output = model(training_input_torch)
    loss = criterion(output, training_output_torch)
    loss.backward()
    optimizer.step()    # Does the update
    if epoch % 10 == 0:
        print('Epoch: {}, Loss: {:.5f}'.format(epoch, loss.item()))

# Now test for rho=17 and 35
rhos = [17, 35]
testing_input = []
testing_output = []

for rho in rhos:
    np.random.seed(123)
    x0 = -15 + 30 * np.random.random((100, 3))

    x_t = np.asarray([integrate.odeint(lorenz_deriv, x0_j, t, args=(sigma, beta, rho)) for x0_j in x0])
    
    for j in range(100):
        testing_input.append(x_t[j,:-1,:])
        testing_output.append(x_t[j,1:,:])

testing_input = np.vstack(testing_input)
testing_output = np.vstack(testing_output)

# Convert to PyTorch tensors
testing_input_torch = torch.tensor(testing_input, dtype=torch.float32)
testing_output_torch = torch.tensor(testing_output, dtype=torch.float32)

# Predict future states
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Do not calculate gradients
    predictions = model(testing_input_torch)

# Calculate the loss between the predictions and the true future states
test_loss = criterion(predictions, testing_output_torch)
print('Test loss:', test_loss.item())




Epoch: 0, Loss: 279.39880
Epoch: 10, Loss: 184.66254
Epoch: 20, Loss: 103.02059
Epoch: 30, Loss: 45.08611
Epoch: 40, Loss: 21.29359
Epoch: 50, Loss: 10.91245
Epoch: 60, Loss: 4.06014
Epoch: 70, Loss: 2.64286
Epoch: 80, Loss: 1.86473
Epoch: 90, Loss: 1.21212
Test loss: 0.6688587069511414


## RNN

In [6]:
# Define the Lorenz system
def lorenz_deriv(x_y_z, t0, sigma=sigma, beta=beta, rho=rho):
    x, y, z = x_y_z
    return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]

# Generate training data for rho=10, 28, 40
rhos = [10, 28, 40]
training_input = []
training_output = []

for rho in rhos:
    np.random.seed(123)
    x0 = -15 + 30 * np.random.random((100, 3))

    x_t = np.asarray([integrate.odeint(lorenz_deriv, x0_j, t, args=(sigma, beta, rho)) for x0_j in x0])
    
    for j in range(100):
        training_input.append(x_t[j,:-1,:])
        training_output.append(x_t[j,1:,:])

training_input = np.vstack(training_input)
training_output = np.vstack(training_output)

# Convert numpy arrays to PyTorch tensors
training_input_torch = torch.tensor(training_input, dtype=torch.float32)
training_output_torch = torch.tensor(training_output, dtype=torch.float32)

class SimpleRNN(nn.Module):
    def __init__(self):
        super(SimpleRNN, self).__init__()
        self.fc1 = nn.Linear(3, 50)  # Input dimension is 3 (for x, y, z)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 3)  # Output dimension is 3 (for x, y, z)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)  # No activation function here because the output can be any real number
    
# Initialize the model and optimizer
model = Net()
optimizer = Adam(model.parameters())

# Define the loss function
criterion = nn.MSELoss()

# Train the network
for epoch in range(100):  # 100 epochs
    optimizer.zero_grad()   # zero the gradient buffers
    output = model(training_input_torch)
    loss = criterion(output, training_output_torch)
    loss.backward()
    optimizer.step()    # Does the update
    if epoch % 10 == 0:
        print('Epoch: {}, Loss: {:.5f}'.format(epoch, loss.item()))

# Now test for rho=17 and 35
rhos = [17, 35]
testing_input = []
testing_output = []

for rho in rhos:
    np.random.seed(123)
    x0 = -15 + 30 * np.random.random((100, 3))

    x_t = np.asarray([integrate.odeint(lorenz_deriv, x0_j, t, args=(sigma, beta, rho)) for x0_j in x0])
    
    for j in range(100):
        testing_input.append(x_t[j,:-1,:])
        testing_output.append(x_t[j,1:,:])

testing_input = np.vstack(testing_input)
testing_output = np.vstack(testing_output)

# Convert to PyTorch tensors
testing_input_torch = torch.tensor(testing_input, dtype=torch.float32)
testing_output_torch = torch.tensor(testing_output, dtype=torch.float32)

# Predict future states
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Do not calculate gradients
    predictions = model(testing_input_torch)

# Calculate the loss between the predictions and the true future states
test_loss = criterion(predictions, testing_output_torch)
print('Test loss:', test_loss.item())




Epoch: 0, Loss: 310.94574
Epoch: 10, Loss: 232.57372
Epoch: 20, Loss: 159.25790
Epoch: 30, Loss: 84.63853
Epoch: 40, Loss: 28.08471
Epoch: 50, Loss: 8.21618
Epoch: 60, Loss: 5.30645
Epoch: 70, Loss: 2.64519
Epoch: 80, Loss: 1.94134
Epoch: 90, Loss: 1.25726
Test loss: 0.6699401140213013


In [7]:
# Define the Lorenz system
def lorenz_deriv(x_y_z, t0, sigma=sigma, beta=beta, rho=rho):
    x, y, z = x_y_z
    return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]

# Generate training data for rho=10, 28, 40
rhos = [10, 28, 40]
training_input = []
training_output = []

for rho in rhos:
    np.random.seed(123)
    x0 = -15 + 30 * np.random.random((100, 3))

    x_t = np.asarray([integrate.odeint(lorenz_deriv, x0_j, t, args=(sigma, beta, rho)) for x0_j in x0])
    
    for j in range(100):
        training_input.append(x_t[j,:-1,:])
        training_output.append(x_t[j,1:,:])

training_input = np.vstack(training_input)
training_output = np.vstack(training_output)

# Convert numpy arrays to PyTorch tensors
training_input_torch = torch.tensor(training_input, dtype=torch.float32)
training_output_torch = torch.tensor(training_output, dtype=torch.float32)

class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=3, hidden_size=20, num_layers=1)
        self.fc = nn.Linear(20, 3)

    def forward(self, x):
        x, _ = self.lstm(x)
        return self.fc(x[-1, :, :])
    
# Initialize the model and optimizer
model = Net()
optimizer = Adam(model.parameters())

# Define the loss function
criterion = nn.MSELoss()

# Train the network
for epoch in range(100):  # 100 epochs
    optimizer.zero_grad()   # zero the gradient buffers
    output = model(training_input_torch)
    loss = criterion(output, training_output_torch)
    loss.backward()
    optimizer.step()    # Does the update
    if epoch % 10 == 0:
        print('Epoch: {}, Loss: {:.5f}'.format(epoch, loss.item()))

# Now test for rho=17 and 35
rhos = [17, 35]
testing_input = []
testing_output = []

for rho in rhos:
    np.random.seed(123)
    x0 = -15 + 30 * np.random.random((100, 3))

    x_t = np.asarray([integrate.odeint(lorenz_deriv, x0_j, t, args=(sigma, beta, rho)) for x0_j in x0])
    
    for j in range(100):
        testing_input.append(x_t[j,:-1,:])
        testing_output.append(x_t[j,1:,:])

testing_input = np.vstack(testing_input)
testing_output = np.vstack(testing_output)

# Convert to PyTorch tensors
testing_input_torch = torch.tensor(testing_input, dtype=torch.float32)
testing_output_torch = torch.tensor(testing_output, dtype=torch.float32)

# Predict future states
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Do not calculate gradients
    predictions = model(testing_input_torch)

# Calculate the loss between the predictions and the true future states
test_loss = criterion(predictions, testing_output_torch)
print('Test loss:', test_loss.item())




Epoch: 0, Loss: 330.02725
Epoch: 10, Loss: 257.17392
Epoch: 20, Loss: 200.10478
Epoch: 30, Loss: 138.83022
Epoch: 40, Loss: 76.49510
Epoch: 50, Loss: 27.27403
Epoch: 60, Loss: 4.38100
Epoch: 70, Loss: 2.60705
Epoch: 80, Loss: 1.35862
Epoch: 90, Loss: 0.90163
Test loss: 0.5337609648704529
